In [5]:
import numpy as np
from itertools import product

class SudokuSolver:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.size = len(puzzle)
        self.box_size = int(self.size ** 0.5)
        self.domains = {i: set(range(1, 10)) for i in product(range(self.size), range(self.size))}
        self.initialize_domains()
        self.constraints = self.get_constraints()
    
    def initialize_domains(self):
        # Initialize domains based on the initial puzzle values
        for i in range(self.size):
            for j in range(self.size):
                if self.puzzle[i][j] != 0:
                    self.domains[(i, j)] = {self.puzzle[i][j]}
    
    def get_constraints(self):
        # Generate constraints for Sudoku puzzle
        constraints = []
        for i, j in product(range(self.size), repeat=2):
            for k in range(self.size):
                if k != j:
                    constraints.append(((i, j), (i, k)))  # Row constraint
                if k != i:
                    constraints.append(((i, j), (k, j)))  # Column constraint
            box_x = i // self.box_size
            box_y = j // self.box_size
            for x, y in product(range(box_x * self.box_size, (box_x + 1) * self.box_size),
                                range(box_y * self.box_size, (box_y + 1) * self.box_size)):
                if (x, y) != (i, j):
                    constraints.append(((i, j), (x, y)))  # Box constraint
        return constraints
    
    def is_consistent(self, cell1, cell2, value1, value2):
        # Check if two cells have different values
        return value1 != value2
    
    def revise(self, cell1, cell2):
        # Revise the domain of cell1 based on the constraints with cell2
        revised = False
        if cell1 != cell2:
            for value1 in self.domains[cell1].copy():
                if all(not self.is_consistent(cell1, cell2, value1, value2) for value2 in self.domains[cell2]):
                    self.domains[cell1].remove(value1)
                    revised = True
        return revised
    
    def solve(self):
        # Iteratively revise the domains until a solution is found or no further progress is possible
        while True:
            revised = False
            for cell1, cell2 in self.constraints:
                if self.revise(cell1, cell2):
                    revised = True
            if not revised:
                break
        
        if all(len(self.domains[cell]) == 1 for cell in self.domains):
            # If all domains have been reduced to single values, return the solution
            return np.array([[list(self.domains[(i, j)])[0] for j in range(self.size)] for i in range(self.size)])
        else:
            # If not all domains are single values, no solution exists
            return None

def print_sudoku(grid):
    for i in range(9):
        if i % 3 == 0 and i != 0:
            print("-"*21)

        for j in range(9):
            if j % 3 == 0 and j != 0:
                print("|", end=" ")

            if j == 8:
                print(grid[i][j])
            else:
                print(str(grid[i][j]) + " ", end="")

initial_puzzle = [
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 1, 0, 2, 9, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 6, 7, 0, 8, 2, 0, 0],
    [0, 0, 2, 6, 0, 9, 5, 0, 0],
    [8, 0, 0, 2, 0, 3, 0, 0, 9],
    [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

solver = SudokuSolver(initial_puzzle)
solution = solver.solve()
if solution is not None:
    print("Solution:")
    print_sudoku(solution)
else:
    print("No solution exists.")


Solution:
4 8 3 | 9 2 1 | 6 5 7
9 6 7 | 3 4 5 | 8 2 1
2 5 1 | 8 7 6 | 4 9 3
---------------------
5 4 8 | 1 3 2 | 9 7 6
7 2 9 | 5 6 4 | 1 3 8
1 3 6 | 7 9 8 | 2 4 5
---------------------
3 7 2 | 6 8 9 | 5 1 4
8 1 4 | 2 5 3 | 7 6 9
6 9 5 | 4 1 7 | 3 8 2
